<a href="https://colab.research.google.com/github/anelglvz/Working-Analyst/blob/main/ML-AI-for-the-Working-Analyst/Semana9_1_Working_Analyst_Explor_Explot_MultiArmedBandit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bandido multibrazo


##Bibliotecas y Funciones


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
maquinas = 4
medias = np.zeros(maquinas)
stds = np.zeros(maquinas)

In [ ]:
def entorno_multi_armed_bandit(maquinas):
    '''
    Creamos el entorno para el problema "multi_armed_bandit" generando aleatoriamente
    la distribución de probabilidad de los premios que otorga cada máquina
    '''
    medias1 = np.random.uniform(-5, 5, size=maquinas)
    stds1 = np.random.uniform(0, 5, size=maquinas)
    return medias1, stds1

In [ ]:
def init_Q(maquinas):
  '''Inicializa el vector Q en ceros, el vector Q representa el valor esperado
   de recompensa de cada máquina'''
  Q =np.zeros(maquinas)
  return Q

In [ ]:
def selecciona_maquina(maquinas):
    '''selecciona una máquina aleatoriamente con distribución unifome'''
    selec = np.random.choice(range(maquinas))
    return selec

In [ ]:
def selecciona_maquina_expl(Q):
    '''selecciona la maquina con el maximo valor de la ganancia esperada'''
    selec = np.argmax(Q)
    return selec

In [ ]:
def selecciona_maquina_egd(maquinas, epsilon):
    '''selecciona una con la estrategia epsilon decreasing greedy'''
    p = np.random.uniform(0,1)

    # cuándo epsilon es pequeño, se escoge la maquina con mayor ganancia
    if p < (1 - epsilon):
        selec = np.argmax(Q)
        return selec  

    # caso contrario, se escoge una al azar
    else: 
        selec = np.random.choice(range(maquinas))
        return selec

In [ ]:
def calcula_recompensa(selec):
  '''calcula la recompensa de jugar en una determinada máquina'''
  r = int(np.random.normal(medias[selec], stds[selec], 1))
  return r

In [ ]:
def actualiza_Q (Q, selec, r, veces_maq):
    '''actualiza el valor esperados de recompensa de la máquina seleccionada'''
    Q[selec] = Q[selec] + 1/(veces_maq[selec])*(r - Q[selec])
    return Q

Lo anterior es: $Q_{k+1}(a)=\frac{1}{k+1} (r_1+\ldots+r_{k+1})$
para una maquina $a$, que es lo mismo a: 
$$ Q_{k+1}(a) = Q_k(a) +\frac{1}{k+1} (r_{k+1}-Q_k(a))  $$

# Introducción

## Definimos nuestro entorno

In [ ]:
### Creamos el entorno definiendo el comportamiento de cada máquina

premio_medio  = np.array([5,  1,  0, -10])
desv_estandar = np.array([1, 0.1, 5,  1])

In [ ]:
# Veamos como se comportarian las máquinas tomando muestreos con dichas mediasy  desviaciones estándar
for maquina in range(4):

  s = np.random.normal(premio_medio[maquina], desv_estandar[maquina],10000)
  plt.xlim(-12,8)
  plt.ylim(0,5)
  plt.hist( s, bins=30,  density=True)
  plt.show()

## ¿Que hace un jugador en el casino?

* inicializamos en cero la variable que guardará los premios o perdidas acumuladas 

In [ ]:
ganado = 0

* seleccionamos una de las maquinas, se juega en ella y se obtiene la recompensa

In [ ]:
selec = 2
r = int(np.random.normal(premio_medio[selec], desv_estandar[selec], 1))
ganado += r
print('recompensa(r):', r)
print('ganado:', ganado)

##  ¿Cómo guardamos la información de los premios obtenidos de cada máquina?

In [ ]:
# Inicializa el vector Q en ceros, donde guardaremos la información
n_maquinas = 4 
Q = init_Q(n_maquinas)
Q

In [ ]:
# Vamos a jugar cierto número de veces (episodios)
episodios = 15
n_maquinas = 4
ganado = 0

veces_maq = np.zeros(n_maquinas)      # guardaremos las veces que se ha jugado por máquina
for ep in range(1, episodios+1):
  selec = np.random.choice([0,1,2,3]) # selección aleatoria -> exploración
  veces_maq[selec] += 1               # actualizamos las veces que se ha jugado x máquina
  r = int(np.random.normal(premio_medio[selec], desv_estandar[selec], 1))
  Q[selec] = Q[selec] + 1/(veces_maq[selec])*(r - Q[selec])
  ganado += r
  print('Episodio_{}, Máquina {}, Premio = {}, Premio_acum = {}, Veces_maq:{} ,Q:{}   \n'.format(ep,
                                                                   selec,
                                                                   r, 
                                                                   ganado,
                                                                   veces_maq,Q.round(2)))


¿Sería correcto si, en lugar de escojer la máquina de forma aleatoria, escojemos la que más rendimientos ha dado?

In [ ]:
Q

In [ ]:
selec = np.argmax(Q)  # selección usando la estrategia de explotación
selec

¿Podemos pensar en una estrategia combinada?

In [ ]:
p = np.random.uniform(0, 1 )  # escojemos un valor entre 0 y 1
ε = .5 # valor que disminuye desde 1 hasta 0 según avance el aprendizaje
print(f'p = {p}, ε = {ε}')
print('¿p es menor que 1 - ε?', p < (1 - ε))
if p < (1 - ε):
    # Explotación
    print('explotación')
    selec = np.argmax(Q)
else: 
    # Exploración
    print('exploración')
    selec = np.random.choice(range(n_maquinas))
selec

Vamos a calcular  ε en función de los episodios, de manera decreciente


In [ ]:
episodios = 100
ε = np.exp(-5 * np.linspace(0, 1, episodios))
ε

In [ ]:
fig =plt.figure()
plt.plot(ε)
plt.xlabel('episodio', fontsize=16)
plt.ylabel('ε', fontsize=16)
fig.suptitle('ε decreciente con el numero de episodios', fontsize=18)

plt.show()

# Aprendizaje con distintas estrategias

In [ ]:
### Definimos el número de máquinas o brazos del problema
n_maquinas = 4

### Creamos el entorno
np.random.seed(4)
medias, stds = entorno_multi_armed_bandit(n_maquinas)  # inicializa la distribución de probabilidad de cada máquina

In [ ]:
### Definimos el número de episodios (juegos)
episodios = 1000

In [ ]:
medias

In [ ]:
stds

## Exploración

In [ ]:
### Inicializamos el vector Q  
Q = init_Q(n_maquinas)  
ganado = 0
veces_maq = np.zeros(n_maquinas)      # guardaremos las veces que se ha jugado por máquina

for i in range(n_maquinas):  # creamos un ciclo para jugar en cada máquina una vez
    veces_maq[i] += 1               # actualizamos las veces que se ha jugado x máquina
    r = calcula_recompensa(i)
    Q = actualiza_Q(Q, i, r, veces_maq)
    ganado += r
    print(Q)

print(veces_maq)

for episodio in range(n_maquinas+1,episodios+1):
    selec = selecciona_maquina(n_maquinas) # elegimos al azar
    veces_maq[selec] += 1               # actualizamos las veces que se ha jugado x máquina
    # Calcula el premio de esa acción
    r = calcula_recompensa(selec)
    # Actualiza la información de lo ganado por cada máquina
    Q = actualiza_Q(Q, selec, r, veces_maq)
  
    ganado += r
  
print("La ganancia total es de: ", ganado)

In [ ]:
veces_maq

## Explotación

In [ ]:
### Inicializamos el vector Q  
Q = init_Q(n_maquinas)  
ganado = 0
veces_maq = np.zeros(n_maquinas)      # guardaremos las veces que se ha jugado por máquina

for i in range(n_maquinas):  # creamos un ciclo para jugar en cada máquina una vez
    veces_maq[i] += 1               # actualizamos las veces que se ha jugado x máquina
    r = calcula_recompensa(i)
    Q = actualiza_Q(Q, i, r, veces_maq)
    ganado += r

print(Q)

for episodio in range(n_maquinas+1 ,episodios+1): # el resto de los episodios los jugamos con una estrategia codiciosa
    selec = selecciona_maquina_expl(Q)
    veces_maq[selec] += 1               # actualizamos las veces que se ha jugado x máquina
    r = calcula_recompensa(selec)
    Q = actualiza_Q(Q, selec, r, veces_maq)
    ganado += r
print("La ganancia total es de: ", ganado)

In [ ]:
veces_maq

## Epsilon decreasing greedy

In [ ]:
# Comando para ver mejor números sin notación científica
np.set_printoptions(suppress=True)

epsilon = 0.1

In [ ]:
### Inicializamos el vector Q  
Q = init_Q(n_maquinas)  
epsilon = np.exp(-5 * np.linspace(0, 1, episodios))  # creamos el vector con un epsilon para cada episodio (No afecta al ser modificado eb linea 16)
ganado = 0
veces_maq = np.zeros(n_maquinas)      # guardaremos las veces que se ha jugado por máquina

for i in range(n_maquinas):  # creamos un ciclo para jugar en cada máquina una vez
    veces_maq[i] += 1               # actualizamos las veces que se ha jugado x máquina
    r = calcula_recompensa(i)
    Q = actualiza_Q(Q, i, r, veces_maq)
    ganado += r

print(veces_maq)

for episodio in range(n_maquinas+1 ,episodios+1):
    #eps = epsilon[episodio-1]  #epsilon decreciente
    eps=0.1
    selec = selecciona_maquina_egd(n_maquinas, eps)

    veces_maq[selec] += 1               # actualizamos las veces que se ha jugado x máquina
    r = calcula_recompensa(selec)
    Q = actualiza_Q(Q, selec, r, veces_maq)
    ganado += r

print("La ganancia total es de: ", ganado)

In [ ]:
veces_maq

# Referencias:

[1]A. Aristizabal, «Understanding Reinforcement Learning Hands-On: Multi-Armed Bandits», Medium, oct. 19, 2020. https://towardsdatascience.com/understanding-reinforcement-learning-hands-on-part-2-multi-armed-bandits-526592072bdc (accedido jul. 30, 2021).

